In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

In [8]:
data=pd.read_excel(r"C:/Users/Dell/Pictures/testnyflip.xlsx")

In [9]:
data.head()

,row_index,content,date,author,Name
0,1,Very effetive for those who are looking for mi...,2020-05-02 00:00:00,Tinni Sen,Chik EggWhite
1,2,I used this for the first time and i have felt...,2020-04-23 00:00:00,Sanjana Puri,Chik EggWhite
2,3,Itâ€™s a nice shampoo and cleanes scalp withou...,2020-03-17 00:00:00,saumya,Chik EggWhite
3,4,I've tried everything for my hair and trust me...,2020-03-16 00:00:00,Priya G,Chik EggWhite
4,5,It makes the hair soft and shiny by preventing...,2020-03-08 00:00:00,Santoshi patruni,Chik EggWhite


In [10]:
#Using spaCy for dependency parsing which forms the crux of aspect extraction
import spacy
from tqdm import tqdm
nlp = spacy.load('C:/Users/Dell/Anaconda3/Lib/site-packages/en_core_web_lg/en_core_web_lg-2.1.0', parse=True, tag=True, entity=True)

In [11]:
#COMPOUND WORDS
competitors = ['marico','P&G','hindustan unilever limited','dabur','petra foods','nestle'] 

In [12]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []

def get_aspect_adj(data):
    for x in tqdm(range(len(data['content']))):
        amod_pairs = []
        advmod_pairs = []
        compound_pairs = []
        xcomp_pairs = []
        neg_pairs = []
        eamod_pairs = []
        eadvmod_pairs = []
        ecompound_pairs = []
        eneg_pairs = []
        excomp_pairs = []
        enemlist = []
        if len(str(data['content'][x])) != 0:
            lines = str(data['content'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
            for line in lines:
                enem_list = []
                for eny in competitors:
                    enem = re.search(eny,line)
                    if enem is not None:
                        enem_list.append(enem.group())
                if len(enem_list)==0:
                    doc = nlp(line)
                    str1=''
                    str2=''
                    for token in doc:
                        if token.pos_ is 'NOUN':
                            for j in token.lefts:
                                if j.dep_ == 'compound':
                                    compound_pairs.append((j.text+' '+token.text,token.text))
                                if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                    str1 = j.text+' '+token.text
                                    amod_pairs.append(j.text+' '+token.text)
                                    for k in j.lefts:
                                        if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                            str2 = k.text+' '+j.text+' '+token.text
                                            amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                    mtch = re.search(re.escape(str1),re.escape(str2))
                                    if mtch is not None:
                                        amod_pairs.remove(str1)
                        if token.pos_ is 'VERB':
                            for j in token.lefts:
                                if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                    advmod_pairs.append(j.text+' '+token.text)
                                if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                    neg_pairs.append(j.text+' '+token.text)
                            for j in token.rights:
                                if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                    advmod_pairs.append(token.text+' '+j.text)
                        if token.pos_ is 'ADJ':
                            for j,h in zip(token.rights,token.lefts):
                                if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                    for k in j.lefts:
                                        if k.dep_ is 'aux':
                                            xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                                elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                    if k.dep_ is 'aux':
                                            neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)

                else:
                    enemlist.append(enem_list)
                    doc = nlp(line)
                    str1=''
                    str2=''
                    for token in doc:
                        if token.pos_ is 'NOUN':
                            for j in token.lefts:
                                if j.dep_ == 'compound':
                                    ecompound_pairs.append((j.text+' '+token.text,token.text))
                                if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                    str1 = j.text+' '+token.text
                                    eamod_pairs.append(j.text+' '+token.text)
                                    for k in j.lefts:
                                        if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                            str2 = k.text+' '+j.text+' '+token.text
                                            eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                    mtch = re.search(re.escape(str1),re.escape(str2))
                                    if mtch is not None:
                                        eamod_pairs.remove(str1)
                        if token.pos_ is 'VERB':
                            for j in token.lefts:
                                if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                    eadvmod_pairs.append(j.text+' '+token.text)
                                if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                    eneg_pairs.append(j.text+' '+token.text)
                            for j in token.rights:
                                if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                    eadvmod_pairs.append(token.text+' '+j.text)
                        if token.pos_ is 'ADJ':
                            for j in token.rights:
                                if j.dep_ is 'xcomp':
                                    for k in j.lefts:
                                        if k.dep_ is 'aux':
                                            excomp_pairs.append(token.text+' '+k.text+' '+j.text)
            pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
            print(pairs)
            epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
            for i in range(len(pairs)):
                if len(compound_pairs)!=0:
                    for comp in compound_pairs:
                        mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                        if mtch is not None:
                            pairs[i] = pairs[i].replace(mtch.group(),comp[0])
            for i in range(len(epairs)):
                if len(ecompound_pairs)!=0:
                    for comp in ecompound_pairs:
                        mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                        if mtch is not None:
                            epairs[i] = epairs[i].replace(mtch.group(),comp[0])

        aspect_terms.append(pairs)
        comp_terms.append(compound_pairs)
        easpect_terms.append(epairs)
        ecomp_terms.append(ecompound_pairs)
        enemy.append(enemlist)
    data['compound_nouns'] = comp_terms
    data['aspect_keywords'] = aspect_terms
    data['competition'] = enemy
    data['competition_comp_nouns'] = ecomp_terms
    data['competition_aspects'] = easpect_terms
    return data

In [13]:
data.shape

(410, 5)

In [14]:
data1 = get_aspect_adj(data)

  0%|                                                                                          | 0/410 [00:00<?, ?it/s]

['mild shampoo']
['first time']
['nice shampoo']
['oily hair']


  1%|▊                                                                                 | 4/410 [00:00<00:13, 30.16it/s]

[]
[]
["n't leave", 'wonderful shampoo', 'certain extent']


  2%|█▍                                                                                | 7/410 [00:00<00:13, 28.86it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 406/410 [00:15<00:00, 24.12it/s]

[]
['normal hair', 'oily hair']
['recently bought']


100%|███████████████████████████████████████████████████████████████████████████████▊| 409/410 [00:15<00:00, 23.35it/s]

['pretty good shampoo', 'good shampoo', 'Work well']


100%|████████████████████████████████████████████████████████████████████████████████| 410/410 [00:15<00:00, 26.34it/s]


In [15]:
my_df = pd.DataFrame(columns = ['row_index','aspect_adj_string','pos_score','neg_score','compound_score','neutral_score'])
my_df

,row_index,aspect_adj_string,pos_score,neg_score,compound_score,neutral_score


In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
my_df = pd.DataFrame(columns = ['row_index','aspect_adj_string','pos_score','neg_score','compound_score','neutral_score'])
global_sentiment = {}
for aspect_adj_tuple in data1.aspect_keywords.iteritems():
    row_sentiment = {}
    for text in aspect_adj_tuple[1]:
        sentiment_array = []
        polarity = analyser.polarity_scores(text)
#         print(polarity)
#         sentiment_array.append(polarity['pos'])
#         sentiment_array.append(polarity['neg'])
#         row_sentiment[text] = sentiment_array
        my_df = my_df.append({'row_index':aspect_adj_tuple[0],'aspect_adj_string':text,'pos_score':polarity['pos'],'neg_score':polarity['neg'],'compound_score':polarity['compound'],'neutral_score':polarity['neu']},ignore_index=True)
#     global_sentiment[aspect_adj_tuple[0]] = row_sentiment
    
# print(global_sentiment)

In [17]:
#Filtering the data to remove the non-sentimental/garbage/gray sentiments from entire dataframe
final_df = my_df[((my_df['compound_score']>0.1) | (my_df['compound_score']< -0.1))]

In [18]:
#verificaction on filtered data
final_df[(final_df['compound_score']<0) & (final_df['pos_score']> final_df['neg_score'])]

,row_index,aspect_adj_string,pos_score,neg_score,compound_score,neutral_score


In [19]:
final_df['final_sentiment'] = ['pos' if i>0 else 'neg' for i in final_df['compound_score']]

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
import spacy

nlp = spacy.load("en_core_web_sm")

for row_txt in final_df.aspect_adj_string.iteritems():
    doc = nlp(row_txt[1])
    adj = []
    aspect = []
    for token in doc:

        if (token.tag_ == 'ADV') or (token.tag_ == 'ADJ') or (token.tag_ == 'UH') or (token.tag_ == 'JJS') or (token.tag_ == 'RB') or (token.tag_ == 'JJ') :
            adj.append(token.text)
            
        elif (token.tag_ == 'NN') or (token.tag_ == 'NNP') or (token.tag_ == 'VBG') or (token.tag_ == 'VB')  or (token.tag_ == 'VBN') or (token.tag_ == 'NNS'):
            aspect.append(token.text)

    if (len(adj) == len(aspect)) & (len(adj)>0):
        final_df.at[row_txt[0],'adj']= adj
        final_df.at[row_txt[0],'aspect']= aspect
    elif len(adj) > len(aspect):
        final_df.at[row_txt[0],'adj']= adj[:len(aspect)]
        final_df.at[row_txt[0],'aspect']= aspect
    elif  len(adj) < len(aspect):
        final_df.at[row_txt[0],'adj']= adj
        final_df.at[row_txt[0],'aspect']= aspect[:len(adj)]
    else:
        final_df.at[row_txt[0],'adj']= adj
        final_df.at[row_txt[0],'aspect']= aspect
            

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
#final_df.to_excel("C:/Users/Dell/Pictures/senti/afteraspectmerged.xlsx")
final_df.head()

,row_index,aspect_adj_string,pos_score,neg_score,compound_score,neutral_score,final_sentiment,adj,aspect
2,2,nice shampoo,0.737,0.000,0.4215,0.263,pos,nice,shampoo
5,6,wonderful shampoo,0.787,0.000,0.5719,0.213,pos,[wonderful],[shampoo]
6,6,certain extent,0.677,0.000,0.2732,0.323,pos,[certain],[extent]
7,8,Nice shampoo,0.737,0.000,0.4215,0.263,pos,[Nice],[shampoo]
10,11,literally damages,0.000,0.744,-0.4404,0.256,neg,[literally],[damages]


In [22]:
thirdreg=final_df[['row_index','aspect_adj_string','final_sentiment']]

In [23]:
initial=pd.read_excel("C:/Users/Dell/Pictures/testnyflip.xlsx")

In [25]:
initial=data[['row_index','content']]

In [26]:
initial.head()

,row_index,content
0,1,Very effetive for those who are looking for mi...
1,2,I used this for the first time and i have felt...
2,3,Itâ€™s a nice shampoo and cleanes scalp withou...
3,4,I've tried everything for my hair and trust me...
4,5,It makes the hair soft and shiny by preventing...


In [27]:
thirdreg.head()
#thirdreg.to_csv("C:/Users/Dell/Pictures/sentily.csv")
subproblem1=pd.merge(initial,thirdreg,on="row_index",how="inner")
subproblem1.head()

,row_index,content,aspect_adj_string,final_sentiment
0,2,I used this for the first time and i have felt...,nice shampoo,pos
1,6,Love it.... It makes hair soft,wonderful shampoo,pos
2,6,Love it.... It makes hair soft,certain extent,pos
3,8,I started using Tressme Botanique shampoo with...,Nice shampoo,pos
4,11,This shampoo doesn't make my hair dry at all i...,literally damages,neg


In [28]:
subproblem1.shape

(365, 4)

In [29]:
geodata_check    = pd.merge(initial,thirdreg["row_index"],on = "row_index",how = "left",indicator = True)
geonew_check     = geodata_check[geodata_check["_merge"] != 'both']
geonew_check.head()

,row_index,content,_merge
0,1,Very effetive for those who are looking for mi...,left_only
2,3,Itâ€™s a nice shampoo and cleanes scalp withou...,left_only
3,4,I've tried everything for my hair and trust me...,left_only
4,5,It makes the hair soft and shiny by preventing...,left_only
7,7,Its wonderful shampoo and conditioner. Doesn't...,left_only


In [31]:
neglected=geonew_check [['row_index','content']]
neglected.head()

,row_index,content
0,1,Very effetive for those who are looking for mi...
2,3,Itâ€™s a nice shampoo and cleanes scalp withou...
3,4,I've tried everything for my hair and trust me...
4,5,It makes the hair soft and shiny by preventing...
7,7,Its wonderful shampoo and conditioner. Doesn't...


In [32]:
neglected.shape

(170, 2)

In [33]:
neglected['content']= neglected['content'].str.replace("[^a-zA-Z#]", " ")
neglected['content']=neglected['content'].str.lower()
neglected['content'] = neglected['content'].str.replace("hair fall", "hairfall ")
neglected = neglected[pd.notnull(neglected['content'])] 



C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [34]:
neglected['content'].sample

<bound method NDFrame.sample of 0      very effetive for those who are looking for mi...
2      it   s a nice shampoo and cleanes scalp withou...
3      i ve tried everything for my hair and trust me...
4      it makes the hair soft and shiny by preventing...
7      its wonderful shampoo and conditioner  doesn t...
9      the bottle is so adorable  nice shampoo and qu...
10     it is really work  after using this shampoo my...
15     gives you lovely hair texture  even without a ...
18     my second bottle  it doesn t stop hairfall  bu...
19     always a fan of chik it makes my hair smooth a...
20     this shampoo is just right product for my hair...
21     it   s good but i don   t suits on my hair  my...
23     reduces hairfall   makes hair shinier and my h...
31     it s have nice mild fragrance makes my hair so...
32     i bought this after trying so many so called h...
41     a mild shampoo with no harsh chemicals clears ...
42     first  its a budget friendly hair product i ha...

In [35]:
#nltk vader lexicon 
final_sentiment=[]
for i in neglected['content'].values:
    try:
        senti=SentimentIntensityAnalyzer()
        analysis=senti.polarity_scores(i)
        final_sentiment.append(analysis['compound'])
    except:
        final_sentiment.append(0)
neglected['final_sentiment']=final_sentiment

pos=neglected[['row_index','content','final_sentiment']][neglected.final_sentiment>0]
neg=neglected[['row_index','content','final_sentiment']][neglected.final_sentiment<0]
hpos=neglected[['row_index','content','final_sentiment']][neglected.final_sentiment>0.75]
hneg=neglected[['row_index','content','final_sentiment']][neglected.final_sentiment<-0.25]

#Converting the polarity values from continuous to categorical
neglected['final_sentiment'][neglected.final_sentiment==0]= 0
neglected['final_sentiment'][neglected.final_sentiment > 0]= 1
neglected['final_sentiment'][neglected.final_sentiment< 0]= -1


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
neglected.head()

,row_index,content,final_sentiment
0,1,very effetive for those who are looking for mi...,0.0
2,3,it s a nice shampoo and cleanes scalp withou...,1.0
3,4,i ve tried everything for my hair and trust me...,1.0
4,5,it makes the hair soft and shiny by preventing...,-1.0
7,7,its wonderful shampoo and conditioner doesn t...,1.0


In [37]:

value = {1.0:'pos',-1.0:'neg',0.0:'neu'} 
neglected.final_sentiment = [value[item] for item in neglected.final_sentiment]  

In [38]:

#neglected.to_csv("C:/Users/Dell/Pictures/neglected1.csv")
neglected.head()


,row_index,content,final_sentiment
0,1,very effetive for those who are looking for mi...,neu
2,3,it s a nice shampoo and cleanes scalp withou...,pos
3,4,i ve tried everything for my hair and trust me...,pos
4,5,it makes the hair soft and shiny by preventing...,neg
7,7,its wonderful shampoo and conditioner doesn t...,pos


In [39]:
#import RAKE
#!pip install rake_nltk
from rake_nltk import Rake
import nltk
#nltk.download('stopwords')
from rake_nltk import Rake
from nltk.corpus import stopwords 
from rake_nltk import Rake
from nltk.corpus import stopwords 

In [40]:

aspect_adj_string =[]
for i in neglected['content'].values:
    try:
        r= Rake()
        r = Rake(min_length=1, max_length=2)
        a=r.extract_keywords_from_text(i)
        b=r.get_ranked_phrases()[:5]
        aspect_adj_string.append(b)
    except:
        aspect_adj_string.append(0)
neglected['aspect_adj_string']=aspect_adj_string


In [41]:
neglected.sample(20)
#neglected.to_csv("C:/Users/Dell/Pictures/aspectcheckwithcrisp25.csv")
#neglected=[['row_index','final_sentiment','aspect_adj_string','Review']]
#thirdreg=[['row_index','Review','final_sentiment','aspect_adj_string']]

,row_index,content,final_sentiment,aspect_adj_string
358,271,i noticed a change in my hair within the three...,pos,"[looks shiny, frizzed free, hair within, hair,..."
413,313,its worth its cost and quite effective in keep...,pos,"[quite effective, bay loved, wrong, worth, vegan]"
148,111,it s a better choice of shampoo for lovely hai...,pos,"[nyle naturals, lovely hair, good packaging, b..."
89,65,time consuming and doesn t clean hair properly,pos,[time consuming]
267,202,it s my favorite shampoo it enhances volume an...,pos,"[favorite shampoo, enhances volume, avoid friz..."
402,307,i brought this shampoo after looking at the re...,neu,"[silky, shampoo, review, looking, brought]"
119,88,makes hair smooth and tangle free hair stays ...,pos,[least days]
186,141,it is paraben free shampoo and it really works...,pos,[manageable]
315,241,can be added as a daily shampoo price is rea...,neu,"[reasonable, added]"
352,266,this is my absolute favorite shampoo i keep c...,pos,[]


In [42]:
subproblem1.shape

(365, 4)

In [43]:
output=subproblem1.append(neglected)
output.sort_values(by='row_index',inplace=True)
output.head()
#output.to_csv("C:/Users/Dell/Pictures/sentiment&aspect25.csv")

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,aspect_adj_string,content,final_sentiment,row_index
0,"[mild shampoo, looking, effetive]",very effetive for those who are looking for mi...,neu,1
0,nice shampoo,I used this for the first time and i have felt...,pos,2
2,"[nice shampoo, liked]",it s a nice shampoo and cleanes scalp withou...,pos,3
3,"[tried everything, shampoo helps, oily hair, s...",i ve tried everything for my hair and trust me...,pos,4
4,"[preventing frizz, hair soft, shiny, makes]",it makes the hair soft and shiny by preventing...,neg,5


In [44]:
output1=output
output.shape

(535, 4)

In [78]:
value = {1:'1.0 out of 5 stars',2:'2.0 out of 5 stars',3:'3.0 out of 5 stars',4:'4.0 out of 5 stars',5:'5.0 out of 5 stars'} 
output.rating = [value[item] for item in output.rating]  

In [79]:
places=['1.0 out of 5 stars','2.0 out of 5 stars']
#xtracting loc and location 
pattern='|'.join(places)
x=output['rating'].str.contains(pattern)
output['final_sentiment']=x.replace((True,False),('negative',output['final_sentiment']))
output= output[pd.notnull(output['content'])] 

output.head()
#output.to_csv("C:/Users/Dell/Pictures/sentiment&aspect(r&d)25.csv")

,aspect_adj_string,content,final_sentiment,rating,row_index
0,[super],super,pos,5.0 out of 5 stars,1
105,[good],good,pos,5.0 out of 5 stars,2
1,"[hair wash, first use, yes, good, found]",yes i felt less hairfall during hair wash i f...,pos,5.0 out of 5 stars,3
0,nice product,Item is not sealed.,negative,2.0 out of 5 stars,4
1,very nice product,Item is not sealed.,negative,2.0 out of 5 stars,4


In [80]:
output.shape

(109, 5)

In [81]:
output.head()

,aspect_adj_string,content,final_sentiment,rating,row_index
0,[super],super,pos,5.0 out of 5 stars,1
105,[good],good,pos,5.0 out of 5 stars,2
1,"[hair wash, first use, yes, good, found]",yes i felt less hairfall during hair wash i f...,pos,5.0 out of 5 stars,3
0,nice product,Item is not sealed.,negative,2.0 out of 5 stars,4
1,very nice product,Item is not sealed.,negative,2.0 out of 5 stars,4


In [45]:
initial=pd.read_excel("C:/Users/Dell/Pictures/code.xlsx")
initial=data[['row_index','content','date','Name']]
initial.head()

,row_index,content,date,Name
0,1,Very effetive for those who are looking for mi...,2020-05-02 00:00:00,Chik EggWhite
1,2,I used this for the first time and i have felt...,2020-04-23 00:00:00,Chik EggWhite
2,3,Itâ€™s a nice shampoo and cleanes scalp withou...,2020-03-17 00:00:00,Chik EggWhite
3,4,I've tried everything for my hair and trust me...,2020-03-16 00:00:00,Chik EggWhite
4,5,It makes the hair soft and shiny by preventing...,2020-03-08 00:00:00,Chik EggWhite


In [ ]:
import pandas as pd
output=pd.read_excel(r"C:/Users/Dell/Pictures/karthika1.xlsx")
output.head()

In [46]:
aspect =[]
for i in output['content'].values:
    try:
        r= Rake()
        r = Rake(min_length=1, max_length=2)
        a=r.extract_keywords_from_text(i)
        b=r.get_ranked_phrases()[:5]
        aspect.append(b)
    except:
        aspect.append(0)
output['aspect']=aspect

In [47]:
output.head()
#output.to_excel(r"C:/Users/Dell/Pictures/karthika2.xlsx")

,aspect_adj_string,content,final_sentiment,row_index,aspect
0,"[mild shampoo, looking, effetive]",very effetive for those who are looking for mi...,neu,1,"[mild shampoo, looking, effetive]"
0,nice shampoo,I used this for the first time and i have felt...,pos,2,"[hair tone, first time, used, good, freezy]"
2,"[nice shampoo, liked]",it s a nice shampoo and cleanes scalp withou...,pos,3,"[nice shampoo, liked]"
3,"[tried everything, shampoo helps, oily hair, s...",i ve tried everything for my hair and trust me...,pos,4,"[tried everything, shampoo helps, oily hair, s..."
4,"[preventing frizz, hair soft, shiny, makes]",it makes the hair soft and shiny by preventing...,neg,5,"[preventing frizz, hair soft, shiny, makes]"


In [48]:
df= pd.merge(output,initial[['row_index','date','Name']],on='row_index', how='inner')

In [49]:
df.head()

,aspect_adj_string,content,final_sentiment,row_index,aspect,date,Name
0,"[mild shampoo, looking, effetive]",very effetive for those who are looking for mi...,neu,1,"[mild shampoo, looking, effetive]",2020-05-02 00:00:00,Chik EggWhite
1,nice shampoo,I used this for the first time and i have felt...,pos,2,"[hair tone, first time, used, good, freezy]",2020-04-23 00:00:00,Chik EggWhite
2,"[nice shampoo, liked]",it s a nice shampoo and cleanes scalp withou...,pos,3,"[nice shampoo, liked]",2020-03-17 00:00:00,Chik EggWhite
3,"[tried everything, shampoo helps, oily hair, s...",i ve tried everything for my hair and trust me...,pos,4,"[tried everything, shampoo helps, oily hair, s...",2020-03-16 00:00:00,Chik EggWhite
4,"[preventing frizz, hair soft, shiny, makes]",it makes the hair soft and shiny by preventing...,neg,5,"[preventing frizz, hair soft, shiny, makes]",2020-03-08 00:00:00,Chik EggWhite


In [50]:
merged8report=df[['row_index','Name','date','content','final_sentiment','aspect']]
#merged8report['date'] = merged8report['date'] .str.replace('Reviewed in India on ', "")
merged8report['final_sentiment'] = merged8report['final_sentiment'] .str.replace('pos', "positive")
merged8report['final_sentiment'] = merged8report['final_sentiment'] .str.replace('neg', "negative")
merged8report['final_sentiment'] = merged8report['final_sentiment'] .str.replace('neu', "neutral")

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [51]:
merged8report.head()

,row_index,Name,date,content,final_sentiment,aspect
0,1,Chik EggWhite,2020-05-02 00:00:00,very effetive for those who are looking for mi...,neutral,"[mild shampoo, looking, effetive]"
1,2,Chik EggWhite,2020-04-23 00:00:00,I used this for the first time and i have felt...,positive,"[hair tone, first time, used, good, freezy]"
2,3,Chik EggWhite,2020-03-17 00:00:00,it s a nice shampoo and cleanes scalp withou...,positive,"[nice shampoo, liked]"
3,4,Chik EggWhite,2020-03-16 00:00:00,i ve tried everything for my hair and trust me...,positive,"[tried everything, shampoo helps, oily hair, s..."
4,5,Chik EggWhite,2020-03-08 00:00:00,it makes the hair soft and shiny by preventing...,negative,"[preventing frizz, hair soft, shiny, makes]"


In [54]:
merged8report.to_excel("C:/Users/Dell/Pictures/testnykaa.xlsx")

In [55]:
report=pd.read_excel("C:/Users/Dell/Pictures/testnykaa.xlsx")
report['text']= report['content']

In [56]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
import nltk
import re
from pywsd.utils import lemmatize_sentence
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

Warming up PyWSD (takes ~10 secs)... took 6.347744941711426 secs.


In [57]:
report['text']= report['text'].str.replace("[^a-zA-Z#]", " ")
lemmatizer = WordNetLemmatizer()
report['text'] = report['text'].apply(lemmatize_sentence)

#report['text']=report['text'].str.lower()
#report['text'] =report['text'].str.split()

In [4]:
#topic modelling
effectonhair=['dandruff','hairfall','hair','reduced','dont','buy','increased','conditioner','conditionerror','shiny','soft',
                'doesnt','support','wash','scalp','sticky','washing','scalps','oily','dirty','grow','hairs','control','washes',
                'smooth','silky','reduces','fall','regrowing','growing','problem','healthy','stopped','variant','decreasing',
                'wash','mild','fall','moisturizing','using','conditioner','moisturizer','smoothens','frizzy','hair','manageable',
                'gradually','improved','type','results','promising','promise','use','life','exotic','lathers','loss','straight',
               'delicate','utilizing','utilize','sleek','dry','tangle','free','satisfied','hairwash','softens','soften','thick',
               'rough','hairs','suits','decreased','reduced','completely','reduce','daily','palm','performed','manage','herbal',
               'parabeen','antihairfall','baldness','shine','real','feel','healthier','healthy','effects','cures','cure','really',
               'working','work','controll','control','stopped','stop','consistency','cooling','refreshing','refresh','slightly',
               'last','years','difference','length','doesnt','literally','paraben','natural','protection','protect','relief','sulfate',
               'sulphate','improve','texture','feels','feel']



In [5]:
packaging=['bottle','damaged','pack','leaked','package','packing','sealed','parcel','slippery','wet','cellophane','tap','tape','packaging',
          'plaster','piece','tightly','fix','cap','leaking','wasted','leakage','condition','opening','covered','lids','approx','ml','seal',
          'damage','open','spilling','lid','defective','spoiled','packet','opened','delivery','packing']

quality=['quality','expected','product','ayurvedic','shampoos','chemicals','chemical','himalaya','patanjali','nyle','chik','dubious',
        'like','shampoo','smelling','bad','good','value','last','ok','never','buy','poor','waste','normal','purchase','didnt','average',
        'improvement','nothing','long','time','fake','worthful','worth','almost','gone','setisfied','satisfied','osm','name','opposite',
         'products','gud','gd','scent','brand','manufacturer','indian','love','worth','experience','nice','works','tried','disappointed',
        'fragrance','best','deal','liked','always','use','super','great','necharal','love','grt','cosmetic','amazing','branded',
        'effective','smell','beautiful','thanks','wonderful','assured','fantastic','favorite','excellent','fine','happy','awesome',
        'super','fav','useful','recommended','recommended','ok']

pricing=['pricing','price','pric','money','rs','rupees','inr','paisa','cheap','worth','charged','pay','mrp','cost']


In [20]:
def matcher(query):
    s=0
    it=0
    wl=0
    tl=0
         

    for x in query: 
        if x in effectonhair:        
            s +=1                    
        if x in packaging:
            it +=1                
        if x in quality:
            wl +=1             
        if x in pricing:
            tl +=1       
        
    max_count=max(s,it,wl,tl)
    print(max_count)
    if max_count==s:
        return 'effectonhair'
    elif max_count==it:
        return 'packaging'
    elif max_count==wl:
        return 'quality'
    elif max_count==tl:
        return 'pricing'
df['class1']=df['content'].apply(matcher)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [18]:
def matcher(query):
    s=0
    it=0
    wl=0
    tl=0
         

    for x in query: 
        if x in effectonhair:        
            s +=1                    
        if x in packaging:
            it +=1                
        if x in quality:
            wl +=1             
        if x in pricing:
            tl +=1       

    
    max_count=max(s,it,wl,tl)
   #print(max_count)
    if max_count1==s:
        return 'effectonhair'
    elif max_count1==it:
        return 'packaging'
    elif max_count1==wl:
        return 'quality'
    elif max_count1==tl:
        return 'pricing'
df['class2']=df['content'].apply(matcher)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [13]:
df.head()

,Unnamed: 0,Unnamed: 0.1,row_index,Name,date,author,rating,content,final_sentiment,Class,aspect,channel,text,class1,class2
0,0,0,0,Nyle anti hairfall,1 December 2018,Amazon Customer,3.0 out of 5 stars,ok product,positive,quality,['ok product'],Amazon,"['ok', 'product']",effectonhair,None
1,1,1,1,Nyle anti hairfall,11 May 2019,Priyesh singh,4.0 out of 5 stars,good,positive,quality,['good'],Amazon,['good'],effectonhair,None
2,2,2,2,Nyle anti hairfall,6 March 2019,Naseer,5.0 out of 5 stars,I have been using this for a year now and hair...,positive,packaging,"['single user', 'long time', 'indian brand', '...",Amazon,"['I', 'have', 'been', 'using', 'this', 'for', ...",effectonhair,None
3,3,3,3,Nyle anti hairfall,23 October 2019,Sonal,5.0 out of 5 stars,"Bought it for INR 152 during the Amazon sale, ...",positive,effect on hair,"['oily scalp', 'inr 152', 'conditioner afterwa...",Amazon,"['Bought', 'it', 'for', 'INR', '152', 'during'...",effectonhair,None
4,4,4,4,Nyle anti hairfall,25 January 2020,Virpalsinh raulji,3.0 out of 5 stars,unexpectedlly one bottle is damaged,negative,packaging,['damaged'],Amazon,"['unexpectedlly', 'one', 'bottle', 'is', 'dama...",effectonhair,None


In [2]:
import pandas as pd
df=pd.read_excel("C:/Users/Dell/Pictures/nylereport.xlsx")

In [6]:
[x for x in df.content.str.split()]

[['ok', 'product'],
 ['good'],
 ['I',
  'have',
  'been',
  'using',
  'this',
  'for',
  'a',
  'year',
  'now',
  'and',
  'hairfall',
  'has',
  'gradually',
  'improved.',
  'This',
  'is',
  'a',
  'big',
  'bottle',
  'and',
  'will',
  'last',
  'you',
  'for',
  'a',
  'LONG',
  'time',
  'for',
  'a',
  'single',
  'user.',
  'This',
  'is',
  'an',
  'Indian',
  'brand',
  'and',
  'manufacturer',
  'and',
  'I',
  'love',
  'the',
  'quality.'],
 ['Bought',
  'it',
  'for',
  'INR',
  '152',
  'during',
  'the',
  'Amazon',
  'sale,',
  'and',
  "it's",
  'worth',
  'every',
  'paisa',
  'I',
  'spent',
  'on',
  'it.',
  'The',
  'scent',
  'is',
  'exotic,',
  'the',
  'shampoo',
  'lathers',
  'easily',
  'and',
  'it',
  'leaves',
  'my',
  'hair',
  'so',
  'soft',
  'and',
  'silky',
  'that',
  'I',
  "don't",
  'need',
  'to',
  'use',
  'a',
  'conditioner',
  'afterward.',
  'Note:',
  'I',
  'have',
  'naturally',
  'straight',
  'hair',
  'and',
  'an',
  'oily',

In [61]:
report.drop_duplicates

<bound method DataFrame.drop_duplicates of      Unnamed: 0  row_index           Name                 date  \
0             0          1  Chik EggWhite  2020-05-02 00:00:00   
1             1          2  Chik EggWhite  2020-04-23 00:00:00   
2             2          3  Chik EggWhite  2020-03-17 00:00:00   
3             3          4  Chik EggWhite  2020-03-16 00:00:00   
4             4          5  Chik EggWhite  2020-03-08 00:00:00   
5             5          6  Chik EggWhite  2020-03-05 00:00:00   
6             6          6  Chik EggWhite  2020-03-05 00:00:00   
7             7          7  Chik EggWhite  2020-02-27 00:00:00   
8             8          8  Chik EggWhite  2020-02-18 00:00:00   
9             9          9  Chik EggWhite  2020-02-12 00:00:00   
10           10         10  Chik EggWhite  2020-02-05 00:00:00   
11           11         11  Chik EggWhite  2020-02-02 00:00:00   
12           12         11  Chik EggWhite  2020-02-02 00:00:00   
13           13         12  Chik 

In [62]:
report.to_excel("C:/Users/Dell/Pictures/testnf.xlsx")

In [ ]:
final8=pd.read_excel("C:/Users/Dell/Pictures/afterclass.xlsx")

In [ ]:
report=pd.read_excel("C:/Users/Dell/Pictures/mouthshut2.xlsx")

In [63]:
report.shape

(532, 9)

In [ ]:
report=report[['row_index','Brand','Name','Variant','date','author','rating','content','final_sentiment','Class','aspect','channel']]

In [ ]:
report.columns

In [ ]:
report.to_excel("C:/Users/Dell/Pictures/mouthshut3.xlsx")

In [397]:
import pandas as pd
df=pd.read_excel("C:/Users/Dell/Pictures/flipchik.xlsx")

In [398]:
df['date1']=1

In [399]:
df['date']=[x for x in df.date.str.replace(","," ")]

In [162]:
#df['date']=[x for x in df.date.str.replace(",","").str.split()]

In [400]:
df.iloc[:10]

,row-index,content,rating,date,author,Name,date1
0,1,super,5,10 months ago,Flipkart Customer,Chik HPEW,1
1,2,good,5,Mar 2019,S nazeefullah S nazeefullah,Chik HPEW,1
2,3,yes i felt less hairfall during hair wash. I f...,5,6 months ago,Dhanishtha Awasthi,Chik HPEW,1
3,4,Item is not sealed.,2,11 months ago,Naveen J,Chik HPEW,1
4,5,It's good my hair become so soft and glossy it...,5,Mar 2019,Flipkart Customer,Chik HPEW,1
5,6,good,5,6 months ago,Flipkart Customer,Chik HPEW,1
6,7,very nice product I like it,5,8 months ago,Ashish Yadav,Chik HPEW,1
7,8,It's so good,5,8 months ago,Shaik Faiyaz,Chik HPEW,1
8,9,best deals,4,Feb 2019,Sandeep jaiswal,Chik HPEW,1
9,10,Good,5,1 month ago,Flipkart Customer,Chik HPEW,1


In [185]:
#df.date[0].split()

In [401]:
one_year_review = df.loc[df['date'].str.contains('^[0-9]+')]

In [402]:
balance_review = df.loc[df['date'].str.contains('^[a-z]|[A-Z]')]

In [406]:
balance_review

,row-index,content,rating,date,author,Name,date1
1,2,good,5,2019-03-01,S nazeefullah S nazeefullah,Chik HPEW,1
4,5,It's good my hair become so soft and glossy it...,5,2019-03-01,Flipkart Customer,Chik HPEW,1
8,9,best deals,4,2019-02-01,Sandeep jaiswal,Chik HPEW,1
91,92,thanks for before time delivery,5,2019-04-01,Flipkart Customer,Chik HPEW,1


In [404]:
balance_review['date'] = balance_review['date1'].astype(str) + ' ' + balance_review['date'].astype(str)
balance_review['date'] = pd.to_datetime(balance_review['date'],format='%d %b %Y')

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [405]:
one_year_review.head(5)

,row-index,content,rating,date,author,Name,date1
0,1,super,5,10 months ago,Flipkart Customer,Chik HPEW,1
2,3,yes i felt less hairfall during hair wash. I f...,5,6 months ago,Dhanishtha Awasthi,Chik HPEW,1
3,4,Item is not sealed.,2,11 months ago,Naveen J,Chik HPEW,1
5,6,good,5,6 months ago,Flipkart Customer,Chik HPEW,1
6,7,very nice product I like it,5,8 months ago,Ashish Yadav,Chik HPEW,1


In [407]:
one_year_review['date']=one_year_review['date'].str.replace("months ago"," ").str.replace("month ago"," ")
one_year_review['month'] = one_year_review['date'].str.strip()

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [408]:
c

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


dtype('<M8[ns]')

In [409]:
one_year_review['month_days'] = one_year_review['month'].astype(int)*30

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [410]:
one_year_review['review_date'] = one_year_review['curr_date'] - pd.to_timedelta(one_year_review['month_days'],unit='D')

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [411]:
one_year_review.iloc[1:3]

,row-index,content,rating,date,author,Name,date1,month,curr_date,month_days,review_date
2,3,yes i felt less hairfall during hair wash. I f...,5,6,Dhanishtha Awasthi,Chik HPEW,1,6,2020-05-07,180,2019-11-09
3,4,Item is not sealed.,2,11,Naveen J,Chik HPEW,1,11,2020-05-07,330,2019-06-12


In [412]:
balance_review.iloc[0:3]

,row-index,content,rating,date,author,Name,date1
1,2,good,5,2019-03-01,S nazeefullah S nazeefullah,Chik HPEW,1
4,5,It's good my hair become so soft and glossy it...,5,2019-03-01,Flipkart Customer,Chik HPEW,1
8,9,best deals,4,2019-02-01,Sandeep jaiswal,Chik HPEW,1


In [413]:
one_year_review=one_year_review.rename({'review_date': 'date','date':'month'},axis=1)
#one_year_review=[['row-index','content','rating','review_date','author','Name']]

In [416]:
data=one_year_review[['row-index','content','rating','date','author','Name']]

In [418]:
data1=balance_review[['row-index','content','rating','date','author','Name']]

In [419]:
df=data.append(data1)

In [422]:
df.sort_index()

,row-index,content,rating,date,author,Name
0,1,super,5,2019-07-12,Flipkart Customer,Chik HPEW
1,2,good,5,2019-03-01,S nazeefullah S nazeefullah,Chik HPEW
2,3,yes i felt less hairfall during hair wash. I f...,5,2019-11-09,Dhanishtha Awasthi,Chik HPEW
3,4,Item is not sealed.,2,2019-06-12,Naveen J,Chik HPEW
4,5,It's good my hair become so soft and glossy it...,5,2019-03-01,Flipkart Customer,Chik HPEW
5,6,good,5,2019-11-09,Flipkart Customer,Chik HPEW
6,7,very nice product I like it,5,2019-09-10,Ashish Yadav,Chik HPEW
7,8,It's so good,5,2019-09-10,Shaik Faiyaz,Chik HPEW
8,9,best deals,4,2019-02-01,Sandeep jaiswal,Chik HPEW
9,10,Good,5,2020-04-07,Flipkart Customer,Chik HPEW


In [1]:
df.to_excel("C:/Users/Dell/Pictures/nyle ADS.xlsx")

NameError: name 'df' is not defined

In [4]:
import pandas as pd
df=pd.read_excel("C:/Users/Dell/Pictures/nyle ADS.xlsx")

In [5]:
df.head()

,content_name,content_date,content_author,Name
0,As mentioned the shampoo has all the natural I...,2019-10-08,Afshan Shayesta,Nyle ADS
1,This is my absolute favorite shampoo? i keep c...,2020-04-06,simrah ali,Nyle ADS
2,This is good for oily hair else oil your hair ...,2020-03-15,Garima Tiwari,Nyle ADS
3,"After using this product, my hair becomes so s...",2020-02-20,S Hussain,Nyle ADS
4,I have used this shampoo for the entire winter...,2020-02-01,Madhurima Ghosh,Nyle ADS


In [6]:
df = df.drop_duplicates(subset=['content_author'])

In [8]:
df.to_excel("C:/Users/Dell/Pictures/NyleADS.xlsx")

In [158]:
len(df)

54